In [1]:
import networkx as nx # version 2.2
import matplotlib.pyplot as plt
import re
import cvxpy as cp
import operator #to sort elements in a list of tuples
import itertools
import math
import numpy as np
import os
import sys
import time

import Init_NetRate as Init
import Cascade_generation_functions_NetRate as Gen

In [2]:
network_file_name = "./../NetInf_mycode/Generation_files/Gen_1/G_1.txt"
cascade_file_name = "./../NetInf_mycode/Generation_files/Gen_1/C_1.txt"
window = 10
beta = 0.5
model = 0

In [9]:
'''
Create the matrices in the cas of the exponential model
'''
def Create_matrices(ground_truth,cascades_DAG_dic):
    num_cascade_per_nodes = np.zeros((1,ground_truth.number_of_nodes()))
    A_pot = np.zeros((ground_truth.number_of_nodes(),ground_truth.number_of_nodes())) # more or less corresponds to Psi 2 in the paper
    A_bad = np.zeros((ground_truth.number_of_nodes(),ground_truth.number_of_nodes())) # more or less corresponds to Psi 1 in the paper
    for c in cascades_DAG_dic :
        DAG = cascades_DAG_dic[c]

        for i in DAG.nodes():
            num_cascade_per_nodes[0,i] += 1
            t_v1 = DAG.nodes[i]["time"]
            parents = list(DAG.predecessors(i))
            if len(parents)==0:
                continue
            for j in parents :
                t_v2 = DAG.nodes[j]["time"]
                A_pot[j,i] += t_v1-t_v2 #Since log(S(t_v2|t_v1;alpha_v1,v2)) = alpha_v1,v2 * (t_v2-t_v1)
                if (t_v1-t_v2)<=0:
                    print("Error delta time neg")
        for node in G_true.nodes() :
            if node not in DAG.nodes() : # Meaning this node was not infected during cascade c
                for node_infected in DAG.nodes():
                    t_infected = DAG.nodes[node_infected]["time"]
                    A_bad[node_infected,node] += window - t_infected 
    return (A_pot,A_bad),num_cascade_per_nodes

def Infer_Network_edges(ground_truth,matrix_list,nb_cascades_per_node,cascades_DAG_dic):
    A_hat = np.zeros((ground_truth.number_of_nodes(),ground_truth.number_of_nodes()))
    A_pot = matrix_list[0]
    A_bad = matrix_list[1]
    for i in ground_truth.nodes():
        print("For node ", i)
        if nb_cascades_per_node[0,i]== 0:
            A_hat[:,i] = 0
            continue
        #Start of cvxnp
        a_hat = cp.Variable(ground_truth.number_of_nodes()) # is alpha_j,i restricted to one column (i.e one node) hence good for // comput.
        t_hat = cp.Variable(int(nb_cascades_per_node[0,i]))
        obj = 0
        constr = []   
        for j in ground_truth.nodes():
            if A_pot[j,i]>0:
                obj = -a_hat[j]*(A_pot[j,i]+A_bad[j,i]) + obj
            elif A_pot[j,i]==0:
                constr.append(a_hat[j]==0)

        c_act = 0
        for c in cascades_DAG_dic :
            tmp = 0
            DAG = DAG_C_dic[c]
            if i in DAG.nodes():
                i_parents = list(DAG.predecessors(i))
                if len(i_parents)>0:
                    for papa in i_parents :
                        tmp = tmp + a_hat[papa]
                    constr.append(t_hat[c_act]==tmp)
                    obj = obj + cp.log(t_hat[c_act])
                    c_act += 1

        constr.append(a_hat>=0)
        objf = cp.Maximize(obj)
        prob = cp.Problem(objf,constr)
        try :
            result = prob.solve()
        except cp.SolverError:
            print("Use another solver")
            result = prob.solve(solver = cp.SCS)
        print(prob.status)   
#         print(a_hat.value)
        A_hat[:,i] = a_hat.value
    return A_hat

In [11]:
G_true = Gen.Generate_random_graph(15,30)
Cascades = Gen.Generate_all_cascades(G_true,-2000,window,model,beta)
Gen.Save_cascade_to_file("Cascade_test.txt",Cascades,G_true)
G_star,DAG_C_dic = Init.Init("Cascade_test.txt")
# G_true = Init.Load_ground_truth(network_file_name)
# G_star, DAG_C_dic = Init.Init(cascade_file_name)

All nodes were read


In [12]:
s_time = time.time()
A_list,nb_c_per_node = Create_matrices(G_true,DAG_C_dic)
alpha_mat = Infer_Network_edges(G_true,A_list,nb_c_per_node,DAG_C_dic)
f_time = time.time()
elapsed_time = f_time-s_time
print(elapsed_time)

For node  0
optimal
For node  1
optimal
For node  2
optimal
For node  3


C:\Users\joach\Anaconda3\lib\site-packages\cvxpy\problems\problem.py:781: RuntimeWarning: overflow encountered in long_scalars
  if self.max_big_small_squared < big*small**2:


Use another solver
WARN: A->p (column pointers) not strictly increasing, column 1133 empty
WARN: A->p (column pointers) not strictly increasing, column 1134 empty
WARN: A->p (column pointers) not strictly increasing, column 1135 empty
WARN: A->p (column pointers) not strictly increasing, column 1136 empty
WARN: A->p (column pointers) not strictly increasing, column 1137 empty
WARN: A->p (column pointers) not strictly increasing, column 1138 empty
WARN: A->p (column pointers) not strictly increasing, column 1139 empty
WARN: A->p (column pointers) not strictly increasing, column 1140 empty
WARN: A->p (column pointers) not strictly increasing, column 1141 empty
WARN: A->p (column pointers) not strictly increasing, column 1142 empty
WARN: A->p (column pointers) not strictly increasing, column 1143 empty
WARN: A->p (column pointers) not strictly increasing, column 1144 empty
WARN: A->p (column pointers) not strictly increasing, column 1145 empty
WARN: A->p (column pointers) not strictly inc

optimal
For node  4


C:\Users\joach\Anaconda3\lib\site-packages\cvxpy\problems\problem.py:782: RuntimeWarning: overflow encountered in long_scalars
  self.max_big_small_squared = big*small**2


Use another solver
WARN: A->p (column pointers) not strictly increasing, column 1937 empty
WARN: A->p (column pointers) not strictly increasing, column 1938 empty
WARN: A->p (column pointers) not strictly increasing, column 1939 empty
WARN: A->p (column pointers) not strictly increasing, column 1940 empty
WARN: A->p (column pointers) not strictly increasing, column 1941 empty
WARN: A->p (column pointers) not strictly increasing, column 1942 empty
WARN: A->p (column pointers) not strictly increasing, column 1943 empty
WARN: A->p (column pointers) not strictly increasing, column 1944 empty
WARN: A->p (column pointers) not strictly increasing, column 1945 empty
WARN: A->p (column pointers) not strictly increasing, column 1946 empty
WARN: A->p (column pointers) not strictly increasing, column 1947 empty
WARN: A->p (column pointers) not strictly increasing, column 1948 empty
WARN: A->p (column pointers) not strictly increasing, column 1949 empty
WARN: A->p (column pointers) not strictly inc

optimal
For node  5
Use another solver
WARN: A->p (column pointers) not strictly increasing, column 1117 empty
WARN: A->p (column pointers) not strictly increasing, column 1118 empty
WARN: A->p (column pointers) not strictly increasing, column 1119 empty
WARN: A->p (column pointers) not strictly increasing, column 1120 empty
WARN: A->p (column pointers) not strictly increasing, column 1121 empty
WARN: A->p (column pointers) not strictly increasing, column 1122 empty
WARN: A->p (column pointers) not strictly increasing, column 1123 empty
WARN: A->p (column pointers) not strictly increasing, column 1124 empty
WARN: A->p (column pointers) not strictly increasing, column 1125 empty
WARN: A->p (column pointers) not strictly increasing, column 1126 empty
WARN: A->p (column pointers) not strictly increasing, column 1127 empty
WARN: A->p (column pointers) not strictly increasing, column 1128 empty
WARN: A->p (column pointers) not strictly increasing, column 1129 empty
WARN: A->p (column pointe

optimal
For node  6
Use another solver
optimal
For node  7
optimal
For node  8
Use another solver
WARN: A->p (column pointers) not strictly increasing, column 1247 empty
WARN: A->p (column pointers) not strictly increasing, column 1248 empty
WARN: A->p (column pointers) not strictly increasing, column 1249 empty
WARN: A->p (column pointers) not strictly increasing, column 1250 empty
WARN: A->p (column pointers) not strictly increasing, column 1251 empty
WARN: A->p (column pointers) not strictly increasing, column 1252 empty
WARN: A->p (column pointers) not strictly increasing, column 1253 empty
WARN: A->p (column pointers) not strictly increasing, column 1254 empty
WARN: A->p (column pointers) not strictly increasing, column 1255 empty
WARN: A->p (column pointers) not strictly increasing, column 1256 empty
WARN: A->p (column pointers) not strictly increasing, column 1257 empty
WARN: A->p (column pointers) not strictly increasing, column 1258 empty
WARN: A->p (column pointers) not stric

optimal
For node  9
optimal
For node  10
Use another solver
optimal
For node  11
optimal
For node  12
Use another solver
WARN: A->p (column pointers) not strictly increasing, column 1463 empty
WARN: A->p (column pointers) not strictly increasing, column 1464 empty
WARN: A->p (column pointers) not strictly increasing, column 1465 empty
WARN: A->p (column pointers) not strictly increasing, column 1466 empty
WARN: A->p (column pointers) not strictly increasing, column 1467 empty
WARN: A->p (column pointers) not strictly increasing, column 1468 empty
WARN: A->p (column pointers) not strictly increasing, column 1469 empty
WARN: A->p (column pointers) not strictly increasing, column 1470 empty
WARN: A->p (column pointers) not strictly increasing, column 1471 empty
WARN: A->p (column pointers) not strictly increasing, column 1472 empty
WARN: A->p (column pointers) not strictly increasing, column 1473 empty
WARN: A->p (column pointers) not strictly increasing, column 1474 empty
WARN: A->p (col

In [16]:
'''
Precision and recall computation
'''
correct = 0
edge_G_star = list(G_star.edges())
G_star.remove_edges_from(edge_G_star)
for i in range(0,G_true.number_of_nodes()):
    for j in range(0,G_true.number_of_nodes()):
        if alpha_mat[i,j] >=1e-3:
            G_star.add_edge(i,j)
            if (i,j) in G_true.edges():
                correct +=1
#                 print("edge ",(i,j))
#                 print("alpha is ",A_hat[i,j])
print("Precision :",correct/G_star.number_of_edges())
print("Recall :",correct/G_true.number_of_edges())
print("Number of correct infered edges : ",correct)
print("Number of edges in G_hat :",G_star.number_of_edges())
print("Number of edges in the true network ",G_true.number_of_edges())

Precision : 1.0
Recall : 1.0
Number of correct infered edges :  30
Number of edges in G_hat : 30
Number of edges in the true network  30


In [22]:
'''
mean absolute error
'''
mae = 0
nb_correcte_edges=0
for edge in G_true.edges():
    if edge in G_star.edges():
        nb_correcte_edges +=1
        true_alpha = G_true.edges[edge]["weight"][0]
        approx_alpha = alpha_mat[edge[0],edge[1]]
        mae += abs(true_alpha-approx_alpha)/true_alpha #normalized mean absolute error
#         print("For edge " + str(edge)+ " the true weight is : " + str(true_alpha))
#         print("For edge " + str(edge)+ " the approx weight is : " + str(approx_alpha))
try :
    mae = mae/nb_correcte_edges
except ZeroDivisionError:
    print ("There is no correct edge")
print("Normalized mean absolute error is : ",mae)

Normalized mean absolute error is :  0.9452468208695451
